In [1]:
import h5py
import numpy as np
import openfermion as of
from openfermion import FermionOperator

lpbliss_hdf5_output_loading_file_path = "data/H4_original.h5"

In [4]:
# Open the HDF5 file
with h5py.File(lpbliss_hdf5_output_loading_file_path, "r") as fid:
    if "BLISS" in fid:
        BLISS_group = fid["BLISS"]

        if "ovec" in BLISS_group:
            print(f"Loading results for BLISS optimization from file: {lpbliss_hdf5_output_loading_file_path}")
            
            # Read arrays from the file
            ovec = BLISS_group["ovec"][:]
            t1 = BLISS_group["t1"]
            t2 = BLISS_group["t2"]            
            t3 = BLISS_group["t3"]
            N = BLISS_group["N"]
            
            # Combine t1, t2, and t3 into t_opt
            t_opt = [t1, t2, t3]
            
            # Load other datasets from BLISS_HAM
            ham = fid["BLISS_HAM"]
            MOL_DATA = fid["BLISS_HAM"]
            h_const = MOL_DATA["h_const"][:]
            h1e = MOL_DATA["obt"][:]
            h2e = MOL_DATA["tbt"][:]
            h3e = MOL_DATA["threebt"][:]

Loading results for BLISS optimization from file: data/H4_original.h5


In [5]:
def construct_one_body_fermion_operator(ferm_op, obt):

    num_orbitals = obt.shape[0]  # assuming h1e is square
    num_spin_orbs = 2 * num_orbitals
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            coefficient = obt[p, q]
            if coefficient != 0.0:
                
                term_alpha_alpha = FermionOperator(((2*p, 1), (2*q, 0)), coefficient)
                ferm_op += term_alpha_alpha
    
                # Beta-beta: (2*p+1, 1) and (2*q+1, 0)
                term_beta_beta = FermionOperator(((2*p+1, 1), (2*q+1, 0)), coefficient)
                ferm_op += term_beta_beta
    
    return ferm_op


def construct_two_body_fermion_operator(ferm_op, tbt):

    num_orbitals = tbt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    coefficient = tbt[p, q, r, s]
                    if coefficient != 0.0:
                        term_1 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_2 = FermionOperator(((2*p + 1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_3 = FermionOperator(((2*p, 1), (2*q, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        term_4 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        
                        ferm_op += term_1 + term_2 + term_3 + term_4
    
    return ferm_op

def construct_three_body_fermion_operator(ferm_op, threebt):
    num_orbitals = threebt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    for m in range(num_orbitals):
                        for n in range(num_orbitals):
                            coefficient = threebt[p, q, r, s, m, n]
                            if coefficient != 0.0:
                                term_1 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_2 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_3 = FermionOperator(((2*p, 1), (2*q, 0), (2*r+1, 1), (2*s+1, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_4 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_5 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r+1, 1), (2*s+1, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_6 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_7 = FermionOperator(((2*p, 1), (2*q, 0), (2*r+1, 1), (2*s+1, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_8 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r+1, 1), (2*s+1, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)


                                ferm_op += term_1 + term_2 + term_3 + term_4 + term_5 + term_6 + term_7 + term_8
    
    return ferm_op

In [6]:
fermion_operator = FermionOperator()
fermion_operator = construct_one_body_fermion_operator(fermion_operator, h1e)
fermion_operator = construct_two_body_fermion_operator(fermion_operator, h2e)
fermion_operator = construct_three_body_fermion_operator(fermion_operator, h3e)

In [7]:
sparse_op = of.get_sparse_operator(fermion_operator)

In [6]:
print(of.get_ground_state(sparse_op))

(-550.4833105556031, array([-2.31292513e-17-1.27935802e-18j, -1.12831289e-16-2.28073128e-17j,
        2.12330731e-17+2.99027714e-17j,  7.50781042e-02+3.14461185e-02j,
        1.03635609e-16+5.40872929e-17j, -1.35515194e-17-6.77718912e-16j,
        1.05954035e-01+4.43783598e-02j,  2.23733970e-17+5.01465547e-16j,
       -8.40404141e-18+4.67886016e-17j, -1.05954035e-01-4.43783598e-02j,
       -1.32858673e-16-2.91018143e-16j,  2.83100681e-16-2.19619017e-16j,
        1.38987217e-01+5.82141561e-02j,  5.66716759e-16+7.01864846e-17j,
        1.93468747e-16+2.04653102e-16j, -8.37009353e-17-8.88195094e-18j,
       -6.22514361e-17+9.56462993e-17j,  9.05628175e-16+6.84401099e-16j,
        1.50590584e-01+6.30741732e-02j, -5.18824079e-16+5.22066054e-16j,
       -1.66148503e-15-2.32767464e-16j,  1.57941642e-16-1.03470849e-16j,
       -9.91803776e-16-4.97487779e-16j, -1.37020214e-16+1.00835072e-16j,
       -2.13222501e-01-8.93072634e-02j, -3.04463499e-16-4.26505000e-16j,
        4.72095580e-16-2.93712

In [9]:
print(of.jw_get_ground_state_at_particle_number(sparse_op, 10))


(-197.51367712855725, array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]))
